In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

In [2]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [3]:
data = pd.read_csv('bitcoin_exchanges.csv')

In [4]:
data.set_index('Time', inplace = True)

In [5]:
data.index = pd.to_datetime(data.index)

In [6]:
data = data[(data.index >= '2015-01-01')]

In [7]:
data.drop(['others', 'cex.io', 'exmo', 'gemini'], axis = 1, inplace = True)

In [9]:
round(data.tail(),2)

,bit-x,bitfinex,bitstamp,coinbase,itbit,kraken
Time,,,,,,
2019-02-23,3980.32,4088.08,3980.96,3981.65,3981.32,3981.16
2019-02-24,3989.83,4085.58,3988.21,3990.23,3989.13,3987.72
2019-02-25,3797.28,3894.66,3791.93,3792.15,3792.57,3792.29
2019-02-26,3800.02,3904.18,3799.00,3799.63,3799.55,3799.98
2019-02-27,3795.25,3900.13,3793.71,3794.46,3794.35,3794.53


In [10]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = True
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [11]:
# Plot all of the BTC exchange prices
df_scatter(data, 'Bitcoin Price (USD) on various exchanges')